In [ ]:
import $file.sparksession
import sparksession._
import spark.implicits._
import org.apache.spark._
import org.apache.spark.sql.{functions => func, _}
import org.apache.spark.sql.types._
import scala.util.Random
import akka.stream.scaladsl.{Flow, Source}
import akka.actor.ActorSystem
import akka.stream.ActorMaterializer
import scala.concurrent.Future
import scala.concurrent.ExecutionContext.Implicits.global

implicit val as = ActorSystem()
implicit val am = ActorMaterializer()

val random = new Random()

spark.read.json("spanish-la-liga/data/*.json")
    .select(
        func.explode(
            func.array_repeat(
                func.struct(
                    $"HomeTeam".alias("home"), 
                    $"AwayTeam".alias("away"), 
                    $"FTHG".alias("homeGoals"), 
                    $"FTAG".alias("awayGoals"), 
                    $"HTHG".alias("homeGoalsHT"), 
                    $"HTAG".alias("awayGoalsHT"),
                    func.current_date().alias("date"),
                    func.lit(random.alphanumeric.take(1000).toString).alias("description")
                ),
                1000
            )
        ).alias("value")
    )
    .selectExpr("value.*")
    .write.mode(SaveMode.Overwrite).parquet("data/la-liga.parquet")

Seq(
    "Andalucía" -> Seq("Betis", "Malaga", "Xerez", "Almeria", "Granada"),
    "C. Valenciana" -> Seq("Villarreal", "Elche"),
    "Catalunya" -> Seq("Espanyol", "Barcelona"),
    "Madrid" -> Seq("Ath Madrid", "Real Madrid", "Vallecano", "Leganes"),
    "Euskadi" -> Seq("Ath Bilbao", "Eibar"),
    "Galiza" -> Seq("Celta", "La Coruna"),
    "Cantabria" -> Seq("Santander"),
    "Aragón" -> Seq("Zaragoza"),
    "Canarias" -> Seq("Tenerife")
).flatMap{
    case (region, teams) => teams.map((_, region))
}.toDF.select(
    $"_1".alias("team"),
    $"_2".alias("region")
)
.write.mode(SaveMode.Overwrite).parquet("data/dictionary.parquet")

# SPARK UI http://localhost:4040/jobs/

In [ ]:
val ligaDF = spark.read.parquet("data/la-liga.parquet") // Eager execution (schema read)
ligaDF.printSchema

In [ ]:
ligaDF.show()

## Declarative

In [ ]:
val homeGoalsPerTeam =
    ligaDF
        .groupBy($"home".alias("team"))
        .agg(func.sum($"homeGoals").alias("goals"))

In [ ]:
homeGoalsPerTeam.explain(true)

In [ ]:
homeGoalsPerTeam.write.mode(SaveMode.Overwrite).parquet("aggregation.parquet")

In [ ]:
spark.read.parquet("aggregation.parquet").show

# Optimizations

## Pushdown filters

In [ ]:
val goalsPerTeam = ligaDF
        .where($"home" =!= "Betis" && $"away" =!= "Betis")
        .select(
            func.explode(
                func.array(
                    func.struct(
                        $"away".alias("team"), 
                        $"awayGoals".alias("goals")
                    ),
                    func.struct(
                        $"home".alias("team"), 
                        $"homeGoals".alias("goals")
                    )
                )
            ).alias("value")
        )
        .selectExpr("value.*")
        .groupBy($"team")
        .agg(func.sum($"goals").alias("goals"))

goalsPerTeam.explain(true)

In [ ]:
run(
    goalsPerTeam
        .write.mode(SaveMode.Overwrite).parquet("data/goals-per-team-df.parquet")
)

In [ ]:
spark.read.parquet("data/goals-per-team-df.parquet").show

## PushDown from inner stage

In [ ]:
spark.read.parquet("data/dictionary.parquet").printSchema

In [ ]:
val threshold = spark.conf.get("spark.sql.autoBroadcastJoinThreshold")
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1)

val teamsMetadata =  spark.read.parquet("data/dictionary.parquet")

run(
    ligaDF
        .join(teamsMetadata, $"home" === $"team")
        .where($"home" === "Betis")
        .write.mode(SaveMode.Overwrite).parquet("data/teams-no-betis-df.parquet")
)

spark.conf.set("spark.sql.autoBroadcastJoinThreshold", threshold)

In [ ]:
spark.read.parquet("data/teams-no-betis-df.parquet").show

In [ ]:
val threshold = spark.conf.get("spark.sql.autoBroadcastJoinThreshold")
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1)
org.apache.spark.sql.catalyst.encoders.OuterScopes.addOuterScope(this) // Necessary to declare case classes

case class Match(
    home: String,
    away: String,
    homeGoals: Option[Long],
    awayGoals: Option[Long],
    homeGoalsHT: Option[Long],
    awayGoalsHT: Option[Long],
    description: String
)

case class TeamDictionary(
    team: String,
    region: String
)

val teamsMetadata =  spark.read.parquet("data/dictionary.parquet")

run(
    ligaDF
        .as[Match]
        .joinWith(teamsMetadata.as[TeamDictionary], $"home" === $"team")
        .filter(_._1.home != "Betis")
        .selectExpr("_1.*", "_2.*")
        .write.mode(SaveMode.Overwrite).parquet("data/teams-no-betis-ds.parquet")
)

spark.conf.set("spark.sql.autoBroadcastJoinThreshold", threshold)

In [ ]:
spark.read.parquet("data/teams-no-betis-ds.parquet").show

## Caching

In [ ]:
def someHardCalculation(v: Long): Long = {
    Thread.sleep(20) // simulates a computationally expensive operation
    v * 10
}

// This transformation takes some time
val computedData = spark.range(0, 1000).map(someHardCalculation(_))

val cachedDS = computedData.cache() // equivalent to computedData.persist(StorageLevel.MEMORY_ONLY)

In [ ]:
cachedDS.explain()

In [ ]:
run(cachedDS.reduce(_ + _))

run(cachedDS.count())

# Not so good
## Caching

In [ ]:
computedData.unpersist()

In [ ]:
cachedDS.explain()

In [ ]:
run(cachedDS.reduce(_ + _))

run(cachedDS.count())

## Hidden side effects

In [ ]:
spark.read.text("data.json")

In [ ]:
spark.read.text("data2.json")

## Encoder generation unchecked at compilation

In [ ]:
org.apache.spark.sql.catalyst.encoders.OuterScopes.addOuterScope(this) // Necessary to declare case classes
import java.util.Date

case class Match(
    home: String,
    away: String,
    homeGoals: Option[Long],
    awayGoals: Option[Long],
    homeGoalsHT: Option[Long],
    awayGoalsHT: Option[Long],
    date: Date
)

def readMatches(): Dataset[Match] = {
    ligaDF.as[Match]
}

In [ ]:
readMatches()

## Some more optimizations could be added to Dataset API

In [ ]:
spark.read.parquet("data/la-liga.parquet")
    .select($"away")

In [ ]:
org.apache.spark.sql.catalyst.encoders.OuterScopes.addOuterScope(this) // Necessary to declare case classes

case class Match(
    home: String,
    away: String,
    homeGoals: Option[Long],
    awayGoals: Option[Long],
    homeGoalsHT: Option[Long],
    awayGoalsHT: Option[Long]
)

ligaDF
    .as[Match]
    .map(_.away)

## Fine-grained parallelism

### Spark naive implementation

In [ ]:
def someHardCalculation(v: Long): Long = {
    Thread.sleep(20) // simulates a computationally expensive operation
    v * 10
}

val serie = List[Long](0, 1, 2, 3, 4, 5, 6, 7, 8, 800)

run(
    serie.toDS
        .map{
            v =>
                v -> 0L.to(v).map(someHardCalculation).sum
        }
        .write.mode(SaveMode.Overwrite).parquet("composition.parquet")
)

In [ ]:
spark.read.parquet("composition.parquet").show

### Akka Stream naive implementation

In [ ]:
runAsync(
    Source(serie)
         .map{
            v =>
                v -> 0L.to(v).map(someHardCalculation).sum
        }
        .runFold(List.empty[(Long, Long)])((l, t) => t :: l)
)

### Akka stream optimized implementation

In [ ]:
def someHardCalculation(v: Long) = Future {
    Thread.sleep(20) // simulates a computationally expensive operation
    v * 10
}

// Empezar por el programa malo

def calculate(v: Long) = 
    Source.fromIterator(() => 0L.to(v).iterator)
        .mapAsync(4)(someHardCalculation)
        .reduce(_ + _)
        .map(res => v -> res)

runAsync(
    Source(serie)
        .flatMapConcat(calculate)
        .runFold(List.empty[(Long, Long)])((l, t) => t :: l)
)

### Spark optimized implementation

In [ ]:
def someHardCalculation(v: Long): Long = {
    Thread.sleep(20) // simulates a computationally expensive operation
    v * 10
}

run(
    serie.toDS
        .flatMap{
            v =>
                0L.to(v).map(i => v -> i)
        }
        .repartition(8)
        .map {
            case (v, i) =>
                v -> someHardCalculation(i)
        }
        .groupByKey(_._1)
        .mapValues(_._2)
        .reduceGroups(_ + _)
        .select($"value".alias("key"), $"ReduceAggregator(bigint)".alias("value"))
        .write.mode(SaveMode.Overwrite).parquet("composition.parquet")
)

In [ ]:
spark.read.parquet("composition.parquet").show